# Homework 12 Solutions

This homework use a dataset containing information about diamonds from [Kaggle](https://www.kaggle.com/shivam2503/diamonds) and can be downloaded directly [here](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/diamonds.csv).  

The columns in the dataset are:
* carat = weight of diamond
* cut = quality of the cut (Fair, Good, Very Good, Premium, Ideal)
* color =  diamond colour, from J (worst) to D (best)
* clarity = a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
* depth = total depth percentage
* table = width of top of diamond relative to widest point
* price = price in US dollars
* x = length in mm
* y = width in mm
* z = depth in mm

### Question 1: Load and clean the data

When loading the data, use the parameter `index_col = 0` in `read_csv()` to tell Pandas that the first column is the index (name of the rows).

Turn the cut, color, and clarity columns into dummy variables.

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import graphviz
 
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

%matplotlib inline
pd.set_option('display.max_columns', None)

In [8]:
diamonds = pd.read_csv("../data/diamonds.csv", index_col = 0)
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [11]:
diamonds2 = pd.get_dummies(diamonds, columns = ["cut", "color","clarity"],drop_first = True)
diamonds2.head()

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
1,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
5,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


### Question 2: Prepare the data for scikit-learn

In this lab, we will try to predict the price of the diamonds using data in all the other columns (the features or independent variables).

a) Create a variable for all the features/independent variables (all columns except price) and a variable for the dependent variable (the price column).

In [12]:
x = diamonds2.drop(columns = "price")
x.head()

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
1,0.23,61.5,55.0,3.95,3.98,2.43,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0.21,59.8,61.0,3.89,3.84,2.31,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0.23,56.9,65.0,4.05,4.07,2.31,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,0.29,62.4,58.0,4.20,4.23,2.63,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
5,0.31,63.3,58.0,4.34,4.35,2.75,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [13]:
y = diamonds2["price"]

2b)  Split the data into training (80%) and testing (20%) datasets.

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

### Question 3:  Fit and evaluate a decision tree

a) Train a decision tree with maximum depth 3 on your training data.

In [17]:
tree3 = DecisionTreeRegressor(max_depth = 3)
tree3.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

3b) Use your decision tree from part (a) to predict the prices on both the training and testing data.

In [18]:
train_preds3 = tree3.predict(x_train)
test_preds3 = tree3.predict(x_test)

3c) What is the mean squared error of the predictions on the training data and on the testing data?  From these mean squared errors, what can you say about how well the model fits the data?

In [20]:
mean_squared_error(train_preds3,y_train)

1991671.2918656582

In [21]:
mean_squared_error(test_preds3,y_test)

2074512.7316131617

The two mean squared errors are very close together (for their magnitudes) so this does not suggest overfitting is happening.  However, we cannot tell if underfitting is happening.

### Question 4: Decision tree

Create an image of your decision tree and include it in your zip file, if it does not appear in your notebook.  Which features are used to predict the price in your decision tree?

**Hint for Jupyter Hub on Lehman360 and webgraphviz.com:**  Display the tree first to see which column numbers it uses, and only convert those columns to the actual column names.   Also, you can get an image of the tree by zooming out on webgraphviz and then taking a screenshot.

In [25]:
dot_data = export_graphviz(tree3, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("diamonds.dot")

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'diamonds.dot'], make sure the Graphviz executables are on your systems' PATH

In [27]:
with open ("diamonds.dot", "r") as fin:
    with open("diamonds_fixed.dot","w") as fout:
        for line in fin.readlines():
            line = line.replace("X[0]","carat")
            line = line.replace("X[4]","y")
            line = line.replace("X[18]","clarity_SI1")
            fout.write(line)

The carat, y, clarity_SI1 features are used to make predictions in this decision tree (for my training data).

### Question 5:   Best model

Try at least 2 other maximum depth values, computing the mean squared error for the training and testing data.  State which values you tried, which model was best, and why.

In [28]:
tree5 = DecisionTreeRegressor(max_depth = 5)
tree5.fit(x_train,y_train)
train_preds5 = tree5.predict(x_train)
test_preds5 = tree5.predict(x_test)
mean_squared_error(train_preds5,y_train)

1436033.8972775657

In [29]:
mean_squared_error(test_preds5,y_test)

1561885.0005164202

In [30]:
tree10 = DecisionTreeRegressor(max_depth = 10)
tree10.fit(x_train,y_train)
train_preds10 = tree10.predict(x_train)
test_preds10 = tree10.predict(x_test)
mean_squared_error(train_preds10,y_train)

627817.8118948551

In [31]:
mean_squared_error(test_preds10,y_test)

805411.728650409

I looked at decisions trees with max depths 3, 5, and 10. The mean squared errors (MSE) for the depth 10 tree were the lowest, but the test MSE was somewhat higher than the training MSE so this might indicate some overfitting.  For the depths 3 and 5 trees, the training and testing MSEs are similar, so there is no overfitting, but the depth 5 tree has lower MSEs than the depth 3 tree.  So the depth 5 tree would be the best model of the 3 tested, but likely a tree with a depth between 5 and 10 is the best model.

### Bonus Question

Fit a linear regression model to the training data, and compute the mean squared error on the training and testing data.  How does this linear regression model compare to your best decision tree?

In [32]:
lin_model = LinearRegression()
lin_model.fit(x_train,y_train)
lin_pred_train = lin_model.predict(x_train)
lin_pred_test = lin_model.predict(x_test)
mean_squared_error(lin_pred_train,y_train)

1279806.8999784132

In [33]:
mean_squared_error(lin_pred_test,y_test)

1264342.0057157355

The training and testing MSEs for the linear regression model are similar, and less than the decision tree models with depths 3 and 5.  The MSEs are not as good as those of the decision tree with depth 10, but there was some indications of overfitting.  So the linear regression model is probably the best one we tried, but a decision tree with depth between 5 and 10 may be better.